<h1>JanataHack E-Commerce Analytics ML Hackathon</h1>
<p>Predicting gender of a user from e-commerce data</p>

<h2>Initial Approach and Progress</h2>
<p>First submission was a Gradient Boost model using only one or two time variables and 'A' and 'B' sparse columns (No of views in category A1,...,A11 and subcategory B01,...,B91).It got a score of 0.851.<br><br>
Then I added sub-subcategory sparse columns(C001,...,C436) and features like Interval, WeekDay and No_of_Views and the score went upto 0.864.<br><br>
Guessed that the first two digits after D in the product code might help and tried adding a few highly correlated ones. Adding all of such features D00,...,D99 took the score to 0.875<br><br>
Tried all digit wise features instead of the above sparse columns 'A','B' and 'C' but didn't help.<br><br>
Adding 4th and 5th digits also improved score a little. Now, changing the model to XGBoost gave a surprisingly good increase in score(0.897).<br><br>
Adding N_Categories variables(and related) and Christmas variable helped (0.904) but NewYear didn't. Removed a few other useless or redundant features too.<br><br>
Increased n_estimators parameter of XGBoost from 100(default) to 1000, 3000 and then 10000 and noticed improvement in the score(0.908,0.922,0.926 respectively).
</p>

<h2>Final Model</h2>
<h4>Won 10th place (92.6% score on Public data, 92.1% score on Private data)</h4>
<p>Given below is the code for the final submission.</p>

<h3>Importing important libraries, models and tools</h3>

In [1]:
import pandas as pd
import numpy as np
import datetime

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier

/home/hithesh/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/hithesh/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


<h3>Loading train data and encoding gender column </h3>

In [26]:
path = '/home/hithesh/Desktop/ML Competitions/ECommerce_Analytics_Vidhya/train_8wry4cB.csv'
df = pd.read_csv(path)
print(df.head())
df['gender'] = df['gender'].apply(lambda x: 0 if x=='female' else 1)
print(df.head())

  session_id       startTime         endTime  \
0     u16159  15/12/14 18:11  15/12/14 18:12   
1     u10253  16/12/14 14:35  16/12/14 14:41   
2     u19037  01/12/14 15:58  01/12/14 15:58   
3     u14556   23/11/14 2:57   23/11/14 3:00   
4     u24295  17/12/14 16:44  17/12/14 16:46   

                                         ProductList  gender  
0  A00002/B00003/C00006/D28435/;A00002/B00003/C00...  female  
1  A00001/B00009/C00031/D29404/;A00001/B00009/C00...    male  
2                       A00002/B00001/C00020/D16944/  female  
3  A00002/B00004/C00018/D10284/;A00002/B00004/C00...  female  
4  A00001/B00001/C00012/D30805/;A00001/B00001/C00...    male  
  session_id       startTime         endTime  \
0     u16159  15/12/14 18:11  15/12/14 18:12   
1     u10253  16/12/14 14:35  16/12/14 14:41   
2     u19037  01/12/14 15:58  01/12/14 15:58   
3     u14556   23/11/14 2:57   23/11/14 3:00   
4     u24295  17/12/14 16:44  17/12/14 16:46   

                                         Pro

<h3>Extracting Day,Month,Hour and Minute from the Date-Time Variables (startTime and endTime)</h3>
Year wasn't extracted because all of the data is from the year 2014.

In [3]:
df['startTime'] = df['startTime'].apply(lambda x:x.replace('/',' '))
df['endTime'] = df['endTime'].apply(lambda x:x.replace('/',' '))

df['Date'] = df['startTime'].apply(lambda x: datetime.datetime(2000+int(x.split(' ')[2]),int(x.split(' ')[1]),int(x.split(' ')[0])))

df['STime'] = df['startTime'].apply(lambda x: 60*int(x.split(' ')[3].split(':')[0]) + int(x.split(' ')[3].split(':')[1]))
df['ETime'] = df['endTime'].apply(lambda x: 60*int(x.split(' ')[3].split(':')[0]) + int(x.split(' ')[3].split(':')[1]))

df['SHour'] = df['startTime'].apply(lambda x: int(x.split(' ')[3].split(':')[0]))
df['EHour'] = df['endTime'].apply(lambda x: int(x.split(' ')[3].split(':')[0]))

df['SMonth'] = df['startTime'].apply(lambda x: int(x.split(' ')[1]))
df['EMonth'] = df['endTime'].apply(lambda x: int(x.split(' ')[1]))

df['SDay'] = df['startTime'].apply(lambda x: int(x.split(' ')[0]))
df['EDay'] = df['endTime'].apply(lambda x: int(x.split(' ')[0]))

df['SMinute'] = df['startTime'].apply(lambda x: int(x.split(' ')[3].split(':')[1]))
df['EMinute'] = df['endTime'].apply(lambda x: int(x.split(' ')[3].split(':')[1]))

<h3>Creating new features from session_id and date and time features (Possibly useful)</h3>

In [4]:
df['SessId'] = df['session_id'].apply(lambda x: int(x.split('u')[1]))

df['Interval'] = (df['ETime'] - df['STime'])*(df['ETime']>=df['STime']) + (2400-df['STime']+df['ETime'])*(df['ETime']<df['STime'])

df['Inactive'] = (df['SDay']!=df['EDay']) 

df['WeekDay'] = df['Date'].apply(lambda x: x.weekday())

df['Christmas'] = (df['SDay'] >= 23)*(df['SDay']<=25)*(df['SMonth']==12)

df['NewYear'] = (df['SDay']>=28)*(df['SDay']<=30)*(df['SMonth']==12)

df['Work'] = df['startTime'].apply(lambda x: 1 if int(x.split(' ')[3].split(':')[0])>=9 and int(x.split(' ')[3].split(':')[0])<=17 and datetime.datetime(2000+int(x.split(' ')[2]),int(x.split(' ')[1]),int(x.split(' ')[2])).weekday()<=4 else 0)

/home/hithesh/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


<h3>Total number of products viewed and products viewed per time spent as features</h3>

In [5]:
prod_list = df['ProductList']

df['No_of_Views'] = pd.Series(np.zeros(len(prod_list)))

for i in range(len(prod_list)):
    string = prod_list[i]
    no_prods = int(string.count('/')/4)
    prod = string.split(';')
    df.loc[i,'No_of_Views']=len(prod)
    
df['Speed'] = (df['No_of_Views'])/(df['Interval']+0.01)

<h3>Creating Sparse features for categories and subcategories A1,...,A11, B01,...B91, C001,...,C436</h3>
Each of these columns indicate the number of products viewed in the respective category.

In [6]:
for m in range(1,437):
    if(m):
        if(m<10):
            df['C00'+str(m)] = pd.Series(np.zeros(len(prod_list)))
        if(m>=10 and m<100):
            df['C0'+str(m)] = pd.Series(np.zeros(len(prod_list)))
        if(m>=100):
            df['C'+str(m)] = pd.Series(np.zeros(len(prod_list)))
#     print(m)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[2][-3:] == '0'+str(m) or prod[j].split('/')[2][-3:] == str(m) or prod[j].split('/')[2][-3:]=='00'+str(m)):
                if(m<10):
                    df.loc[i,'C00'+str(m)] += 1
                if(m>=10 and m<100):
                    df.loc[i,'C0'+str(m)] += 1
                if(m>=100):
                    df.loc[i,'C'+str(m)] += 1

for l in range(1,92):
    if(l):
        if(l>=10):
            df['B'+str(l)] = pd.Series(np.zeros(len(prod_list)))
        if(l<10):
            df['B0'+str(l)] = pd.Series(np.zeros(len(prod_list)))
#     print(l)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[1][-2:] == '0'+str(l) or prod[j].split('/')[1][-2:] == str(l)):
                if(l<10):
                    df.loc[i,'B0'+str(l)] += 1
                if(l>=10):
                    df.loc[i,'B'+str(l)] += 1
            
    
for k in range(1,12):
    if(k>=10):
        df['A'+str(k)] = pd.Series(np.zeros(len(prod_list)))
    else:
        df['A0'+str(k)] = pd.Series(np.zeros(len(prod_list)))
#     print(k)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[0][-2:] == '0'+str(k) or prod[j].split('/')[0][-2:] == str(k)):
                if(k<10):
                    df.loc[i,'A0'+str(k)] += 1
                else:
                    df.loc[i,'A'+str(k)] += 1

<h3>More Sparse Features using the Product Code</h3>
 Third digit after 'D',4th and 5th digits after 'D', 1st and 2nd digits after 'D'

In [7]:
for p in range(0,10):
    df['E'+str(p)] = pd.Series(np.zeros(len(prod_list)))
#     print(p)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[3][3] == str(p)):
                df.loc[i,'E'+str(p)] += 1
                    
for o in range(0,100):
    if(o>-1):
        if(o<10):
            df['F0'+str(o)] = pd.Series(np.zeros(len(prod_list)))
        if(o>=10):
            df['F'+str(o)] = pd.Series(np.zeros(len(prod_list)))
#     print(o)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[3][4:6] == '0' +str(o) or prod[j].split('/')[3][4:6] == str(o)):
                if(o<10):
                    df.loc[i,'F0'+str(o)] += 1
                if(o>=10):
                    df.loc[i,'F'+str(o)] += 1    
    
for n in range(0,100):
    if(n>-1):
        if(n<10):
            df['D0'+str(n)] = pd.Series(np.zeros(len(prod_list)))
        if(n>=10):
            df['D'+str(n)] = pd.Series(np.zeros(len(prod_list)))
#     print(n)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[3][1:3] == '0'+str(n) or prod[j].split('/')[3][1:3] == str(n)):
                if(n<10):
                    df.loc[i,'D0'+str(n)] += 1
                if(n>=10):
                    df.loc[i,'D'+str(n)] += 1

<h3>Features indicating number of categories and subcategories viewed, per total product views and per time spent</h3>

In [8]:
df['N_CategoriesA'] = pd.Series(np.zeros(len(prod_list)))
for i in range(1,12):
    if(i<10):
        df['N_CategoriesA'] += df['A0'+str(i)]>0
    else:
        df['N_CategoriesA'] += df['A'+str(i)]>0
df['Cat_PropA'] = df['N_CategoriesA']/df['No_of_Views']
df['Cat_SpeedA'] = df['N_CategoriesA']/(df['Interval']+0.01)

df['N_CategoriesB'] = pd.Series(np.zeros(len(prod_list)))
for i in range(1,92):
    if(i<10):
        df['N_CategoriesB'] += df['B0'+str(i)]>0
    else:
        df['N_CategoriesB'] += df['B'+str(i)]>0       
df['Cat_PropB'] = df['N_CategoriesB']/df['No_of_Views']
df['Cat_SpeedB'] = df['N_CategoriesB']/(df['Interval']+0.01)

df['N_CategoriesC'] = pd.Series(np.zeros(len(prod_list)))
for i in range(1,437):
    if(i>=100):
        df['N_CategoriesC'] += df['C'+str(i)]>0
    elif(i>=10):
        df['N_CategoriesC'] += df['C0'+str(i)]>0
    else:
        df['N_CategoriesC'] +=df['C00'+str(i)]>0     
df['Cat_PropC'] = df['N_CategoriesC']/df['No_of_Views']
df['Cat_SpeedC'] = df['N_CategoriesC']/(df['Interval']+0.01)

<h3>Saving the dataframe with new features</h3>
Note: I used two python files initially (combined them now), one for feature engineering and other to test the model. So this step was only to facilitate that.

In [9]:
df=df.drop(['ProductList'],axis=1)
df.to_csv('/home/hithesh/Desktop/ML Competitions/ECommerce_Analytics_Vidhya/mod_df.csv')

<h3>Loading test data</h3>

In [10]:
path = '/home/hithesh/Desktop/ML Competitions/ECommerce_Analytics_Vidhya/test_Yix80N0.csv'
df = pd.read_csv(path)
print(df.head())

  session_id       startTime         endTime  \
0     u12112  08/12/14 13:36  08/12/14 13:36   
1     u19725  19/12/14 13:52  19/12/14 13:52   
2     u11795  01/12/14 10:44  01/12/14 10:44   
3     u22639  08/12/14 20:19  08/12/14 20:22   
4     u18034  15/12/14 19:33  15/12/14 19:33   

                                         ProductList  
0                       A00002/B00003/C00006/D19956/  
1                       A00002/B00005/C00067/D02026/  
2                       A00002/B00002/C00004/D12538/  
3  A00002/B00003/C00079/D22781/;A00002/B00003/C00...  
4                       A00002/B00001/C00010/D23419/  


<h3>Extracting Day,Month,Hour and Minute from the Date-Time Variables (startTime and endTime) - For test data</h3>
Year wasn't extracted because all of the data is from the year 2014.

In [11]:
df['startTime'] = df['startTime'].apply(lambda x:x.replace('/',' '))
df['endTime'] = df['endTime'].apply(lambda x:x.replace('/',' '))

df['Date'] = df['startTime'].apply(lambda x: datetime.datetime(2000+int(x.split(' ')[2]),int(x.split(' ')[1]),int(x.split(' ')[0])))

df['STime'] = df['startTime'].apply(lambda x: 60*int(x.split(' ')[3].split(':')[0]) + int(x.split(' ')[3].split(':')[1]))
df['ETime'] = df['endTime'].apply(lambda x: 60*int(x.split(' ')[3].split(':')[0]) + int(x.split(' ')[3].split(':')[1]))

df['SHour'] = df['startTime'].apply(lambda x: int(x.split(' ')[3].split(':')[0]))
df['EHour'] = df['endTime'].apply(lambda x: int(x.split(' ')[3].split(':')[0]))

df['SMonth'] = df['startTime'].apply(lambda x: int(x.split(' ')[1]))
df['EMonth'] = df['endTime'].apply(lambda x: int(x.split(' ')[1]))

df['SDay'] = df['startTime'].apply(lambda x: int(x.split(' ')[0]))
df['EDay'] = df['endTime'].apply(lambda x: int(x.split(' ')[0]))

df['SMinute'] = df['startTime'].apply(lambda x: int(x.split(' ')[3].split(':')[1]))
df['EMinute'] = df['endTime'].apply(lambda x: int(x.split(' ')[3].split(':')[1]))

<h3>Creating new features from session_id and date and time features (Possibly useful) - For test data</h3>

In [12]:
df['SessId'] = df['session_id'].apply(lambda x: int(x.split('u')[1]))

df['Interval'] = (df['ETime'] - df['STime'])*(df['ETime']>=df['STime']) + (2400-df['STime']+df['ETime'])*(df['ETime']<df['STime'])

df['Inactive'] = (df['SDay']!=df['EDay']) 

df['WeekDay'] = df['Date'].apply(lambda x: x.weekday())

df['Christmas'] = (df['SDay'] >= 23)*(df['SDay']<=25)*(df['SMonth']==12)

df['NewYear'] = (df['SDay']>=28)*(df['SDay']<=30)*(df['SMonth']==12)

df['Work'] = df['startTime'].apply(lambda x: 1 if int(x.split(' ')[3].split(':')[0])>=9 and int(x.split(' ')[3].split(':')[0])<=17 and datetime.datetime(2000+int(x.split(' ')[2]),int(x.split(' ')[1]),int(x.split(' ')[2])).weekday()<=4 else 0)

<h3>Total number of products viewed and products viewed per time spent as features - For test data</h3>

In [13]:
prod_list = df['ProductList']

df['No_of_Views'] = pd.Series(np.zeros(len(prod_list)))

for i in range(len(prod_list)):
    string = prod_list[i]
    no_prods = int(string.count('/')/4)
    prod = string.split(';')
    df.loc[i,'No_of_Views']=len(prod)

df['Speed'] = (df['No_of_Views'])/(df['Interval']+0.01)


<h3>Creating Sparse features for categories and subcategories A1,...,A11, B01,...B91, C001,...,C436 - For test data</h3>
Each of these columns indicate the number of products viewed in the respective category.

In [14]:
for m in range(1,437):
    if(m):
        if(m<10):
            df['C00'+str(m)] = pd.Series(np.zeros(len(prod_list)))
        if(m>=10 and m<100):
            df['C0'+str(m)] = pd.Series(np.zeros(len(prod_list)))
        if(m>=100):
            df['C'+str(m)] = pd.Series(np.zeros(len(prod_list)))
#     print(m)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[2][-3:] == '0'+str(m) or prod[j].split('/')[2][-3:] == str(m) or prod[j].split('/')[2][-3:]=='00'+str(m)):
                if(m<10):
                    df.loc[i,'C00'+str(m)] += 1
                if(m>=10 and m<100):
                    df.loc[i,'C0'+str(m)] += 1
                if(m>=100):
                    df.loc[i,'C'+str(m)] += 1

for l in range(1,92):
    if(l):
        if(l>=10):
            df['B'+str(l)] = pd.Series(np.zeros(len(prod_list)))
        if(l<10):
            df['B0'+str(l)] = pd.Series(np.zeros(len(prod_list)))
#     print(l)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[1][-2:] == '0'+str(l) or prod[j].split('/')[1][-2:] == str(l)):
                if(l<10):
                    df.loc[i,'B0'+str(l)] += 1
                if(l>=10):
                    df.loc[i,'B'+str(l)] += 1
            
    
for k in range(1,12):
    if(k>=10):
        df['A'+str(k)] = pd.Series(np.zeros(len(prod_list)))
    else:
        df['A0'+str(k)] = pd.Series(np.zeros(len(prod_list)))
#     print(k)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[0][-2:] == '0'+str(k) or prod[j].split('/')[0][-2:] == str(k)):
                if(k<10):
                    df.loc[i,'A0'+str(k)] += 1
                else:
                    df.loc[i,'A'+str(k)] += 1

<h3>More Sparse Features using the Product Code - For test data</h3>
 Third digit after 'D',4th and 5th digits after 'D', 1st and 2nd digits after 'D'

In [15]:
for p in range(0,10):
    df['E'+str(p)] = pd.Series(np.zeros(len(prod_list)))
#     print(p)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[3][3] == str(p)):
                df.loc[i,'E'+str(p)] += 1
                
for o in range(0,100):
    if(o>-1):
        if(o<10):
            df['F0'+str(o)] = pd.Series(np.zeros(len(prod_list)))
        if(o>=10):
            df['F'+str(o)] = pd.Series(np.zeros(len(prod_list)))
#     print(o)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[3][4:6] == '0' +str(o) or prod[j].split('/')[3][4:6] == str(o)):
                if(o<10):
                    df.loc[i,'F0'+str(o)] += 1
                if(o>=10):
                    df.loc[i,'F'+str(o)] += 1    
    
for n in range(0,100):
    if(n>-1):
        if(n<10):
            df['D0'+str(n)] = pd.Series(np.zeros(len(prod_list)))
        if(n>=10):
            df['D'+str(n)] = pd.Series(np.zeros(len(prod_list)))
#     print(n)
    for i in range(len(prod_list)):
        string = prod_list[i]
        no_prods = int(string.count('/')/4)
        prod = string.split(';', no_prods)
        for j in range(no_prods):
            if(prod[j].split('/')[3][1:3] == '0'+str(n) or prod[j].split('/')[3][1:3] == str(n)):
                if(n<10):
                    df.loc[i,'D0'+str(n)] += 1
                if(n>=10):
                    df.loc[i,'D'+str(n)] += 1
 

<h3>Features indicating number of categories and subcategories viewed, per total product views and per time spent - for test data</h3>

In [16]:
df['N_CategoriesA'] = pd.Series(np.zeros(len(prod_list)))
for i in range(1,12):
    if(i<10):
        df['N_CategoriesA'] += df['A0'+str(i)]>0
    else:
        df['N_CategoriesA'] += df['A'+str(i)]>0
df['Cat_PropA'] = df['N_CategoriesA']/df['No_of_Views']
df['Cat_SpeedA'] = df['N_CategoriesA']/(df['Interval']+0.01)

df['N_CategoriesB'] = pd.Series(np.zeros(len(prod_list)))
for i in range(1,92):
    if(i<10):
        df['N_CategoriesB'] += df['B0'+str(i)]>0
    else:
        df['N_CategoriesB'] += df['B'+str(i)]>0       
df['Cat_PropB'] = df['N_CategoriesB']/df['No_of_Views']
df['Cat_SpeedB'] = df['N_CategoriesB']/(df['Interval']+0.01)

df['N_CategoriesC'] = pd.Series(np.zeros(len(prod_list)))
for i in range(1,437):
    if(i>=100):
        df['N_CategoriesC'] += df['C'+str(i)]>0
    elif(i>=10):
        df['N_CategoriesC'] += df['C0'+str(i)]>0
    else:
        df['N_CategoriesC'] +=df['C00'+str(i)]>0     
df['Cat_PropC'] = df['N_CategoriesC']/df['No_of_Views']
df['Cat_SpeedC'] = df['N_CategoriesC']/(df['Interval']+0.01)

<h3>Saving the dataframe with new features - for Test Data</h3>
Note: I used two python files initially (combined them now), one for feature engineering and other to test the model. So this step was only to facilitate that.

In [17]:
df=df.drop(['ProductList'],axis=1)
df.to_csv('/home/hithesh/Desktop/ML Competitions/ECommerce_Analytics_Vidhya/mod_df_test.csv')

<h3>Loading the dataframe with new features and Dropping Useless or Redundant Features</h3>
Note: I used two python files initially (combined them now), one for feature engineering and other to test the model. So this step was only to facilitate that. Redundant features were identified using cross validation.

In [18]:
df = pd.read_csv('/home/hithesh/Desktop/ML Competitions/ECommerce_Analytics_Vidhya/mod_df.csv')
print(df.head())
features = df.columns
features = features.drop(['session_id','startTime','endTime','gender','Unnamed: 0','Date','STime','ETime','NewYear','SMinute','EMinute','Cat_SpeedA','Cat_SpeedB','Cat_SpeedC'])
X = df[features]
y = df['gender']

   Unnamed: 0 session_id       startTime         endTime  gender        Date  \
0           0     u16159  15 12 14 18:11  15 12 14 18:12       0  2014-12-15   
1           1     u10253  16 12 14 14:35  16 12 14 14:41       1  2014-12-16   
2           2     u19037  01 12 14 15:58  01 12 14 15:58       0  2014-12-01   
3           3     u14556   23 11 14 2:57   23 11 14 3:00       0  2014-11-23   
4           4     u24295  17 12 14 16:44  17 12 14 16:46       1  2014-12-17   

   STime  ETime  SHour  EHour  ...  D99  N_CategoriesA  Cat_PropA  Cat_SpeedA  \
0   1091   1092     18     18  ...  0.0            1.0   0.250000    0.990099   
1    875    881     14     14  ...  0.0            1.0   0.142857    0.166389   
2    958    958     15     15  ...  0.0            1.0   1.000000  100.000000   
3    177    180      2      3  ...  0.0            1.0   0.333333    0.332226   
4   1004   1006     16     16  ...  0.0            1.0   0.500000    0.497512   

   N_CategoriesB  Cat_PropB  Cat

<h3>Cross Validation by Splitting Training Data</h3>
Models like SVM, GradientBoost, Random Forests, XGBoost were tried with adjusting various parameters (max_depth,min_samples_split,learning_rate etc) and evaluated. XGBoost Classifier with n_estimators 10000 gave the best Cross Validation score. Could have done better with the overfitting, but n_estimators 10000 took a lot of time to execute and it was nearing competition closing time.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# model = SVC()
# model = GradientBoostingClassifier()
# model = RandomForestClassifier()
model = XGBClassifier(n_estimators = 10000)
model.fit(X_train, y_train)
print('Train_Error:',((model.predict(X_train) - y_train) == 0).sum()*100/len(y_train))
print('CV_Error:',((model.predict(X_test) - y_test) == 0).sum()*100/len(y_test),'\n')

Train_Error: 100.0
CV_Error: 91.71428571428571 



<h3>Fitting the model to Full Training Data</h3>

In [22]:
model.fit(X,y)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=10000, n_jobs=0, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)

<h3>Predicting on the Test Set</h3>

In [23]:
df_test = pd.read_csv('/home/hithesh/Desktop/ML Competitions/ECommerce_Analytics_Vidhya/mod_df_test.csv')
features = df_test.columns
sess_id = df_test['session_id']
features = features.drop(['session_id','startTime','endTime','Unnamed: 0','Date','STime','ETime','NewYear','SMinute','EMinute','Cat_SpeedA','Cat_SpeedB','Cat_SpeedC'])
X_test = df_test[features]
pred = model.predict(X_test)

<h3>Converting encoded predicted genders back to male,female and saving the final submission file</h3>

In [25]:
df_test['gender'] = pd.Series(pred).apply(lambda x:'male' if x==1 else 'female')
final = pd.concat([sess_id,df_test['gender']],axis=1)

final.to_csv('/home/hithesh/Desktop/ML Competitions/ECommerce_Analytics_Vidhya/ecomm_sub.csv',header=True , index=False)